#### Detect model from Yu, et al
Implementation of the detection model from the Yu, et al [paper](https://drive.google.com/file/d/1nYl4w41CAcj8XwTEdVwcD5lVheUFIHVy/view?usp=sharing)

Difference(s) from paper:
- Resized to 224x224 with n=2 morphological thinning applied (no median filter)

In [1]:
!pip install pickle5

In [2]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, callbacks

import os
import time
import numpy as np
import pandas as pd

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [3]:
tf.__version__

'2.3.4'

In [4]:
tf.config.list_physical_devices(device_type=None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# specify variables for model
path = 'processed_data/WM-clean224-thin2'
result_path = 'results'
#model_path = '../saved_models'

filename = 'WM-clean224-thin2'
option = '-detund'
map_column = 'thinMap2'

model_id = 'yudetect'
data_id = '224'
note = '-thin2' # -optional

In [ ]:
# load train, dev, and test sets
# directly from S3
start = time.time()

train_key = f'{path}/{filename}-train{option}.pkl'
dev_key = f'{path}/{filename}-dev.pkl'
test_key = f'{path}/{filename}-test.pkl'

train = pickle.loads(my_bucket.Object(key=train_key).get()['Body'].read())
dev = pickle.loads(my_bucket.Object(key=dev_key).get()['Body'].read())
test = pickle.loads(my_bucket.Object(key=test_key).get()['Body'].read())

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f"Train: {len(train)}")
print(f"Dev: {len(dev)}")
print(f"Test: {len(test)}")

print(f"Sanity check: {np.unique(train[map_column][0])}")

#### Quick EDA

In [ ]:
# baseline accuracy of test set
nones = len(test[test.detectLabels == 0])
total = len(test)
print(f"Baseline accuracy: {nones/total*100:.2f}%")

In [ ]:
# train failure type distribution
helper.defect_distribution(train, note='Train Set', mode='detect')

In [ ]:
# dev failure type distribution
helper.defect_distribution(dev, note='Dev Set', mode='detect')

In [ ]:
# test failure type distribution
helper.defect_distribution(test, note='Test Set', mode='detect')

#### Data set-up

In [ ]:
# prepare inputs
start = time.time()

x_train = np.stack(train[map_column])
x_val = np.stack(dev[map_column])
x_test = np.stack(test[map_column])

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: (#rows, xdim, ydim)
print(x_train.shape)

In [ ]:
# expand tensor and create dummy dimension at axis 3
# images in greyscale, so no channel dimension
start = time.time()

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_val = tf.expand_dims(x_val, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_train.shape

In [ ]:
# prepare labels for supervised learning
# note: make sure labels are integers if using sparse categorical cross entropy
start = time.time()

y_train = np.asarray(train['detectLabels']).astype(np.uint8)
y_val = np.asarray(dev['detectLabels']).astype(np.uint8)
y_test = np.asarray(test['detectLabels']).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
type(y_train[0])

#### Model

In [ ]:
# define model architecture

model = models.Sequential()
model.add(layers.Input(shape=(224, 224, 1)))
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'), input_shape=x_train.shape[1:])
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

In [ ]:
# set model optimizer and metrics
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# run model
start = time.time()

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=10)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# visualize accuracy and loss history
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

#### Model results

In [ ]:
# # save model to local instance
# model.save(f'{model_path}/{model_id}-{data_id}{note}')

In [ ]:
# save model to S3
s3_save_keras_model(model, f'{model_id}-{data_id}{note}')

In [ ]:
# compute model results on test set
start = time.time()
results = model.evaluate(x_test, y_test)
print("Wall time: {:.2f} seconds".format(time.time() - start))
print(results)

In [ ]:
# generate predictions for model analysis
start = time.time()
y_pred = model.predict(x_test)
y_max = np.argmax(y_pred, axis=1).astype(np.uint8)
predictions = [y_max, y_pred]
print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# save predictions
# save to local instance
with open(f'{result_path}/{model_id}-{data_id}{note}.pkl', "wb") as f:
    pickle.dump(predictions, f)

In [ ]:
# plot confusion matrix
helper.plot_confusion_matrix(y_test, y_max, mode='detect', normalize=True)

In [ ]:
# plot confusion matrix counts
helper.plot_confusion_matrix(y_test, y_max, mode='detect', normalize=False)